In [36]:
import re
from pathlib import Path
import pandas as pd

data_folder = Path("C:/Users/Jing Li/Desktop/cv2db/project/Inputs")
data_to_open = data_folder / "Resume_B.txt"

In [37]:
with open(data_to_open) as f: 
	data = f.read()
    
# Seperate each person
pattern = r"Qualifications and Experience"
temp = list(filter(None, re.split(pattern, data)))
del temp[0]

In [38]:
# seperate QE

pattern_QE = r"\n\d+\. "
# test = [re.split(pattern_QE, i) for i in temp]
QEs = [re.split(pattern_QE, i)[0] for i in temp]
data = [re.split(pattern_QE, i)[1] for i in temp]

In [39]:
#Checking
# print(len(QE), QE)
# print(len(data), data[0])

In [40]:
# Split: name and title

name_title = [i.split("\n", 1)[0] for i in data]
ppls_noname = [i.split("\n", 1)[1] for i in data]

name_title = [i.replace(" Summary", "") for i in name_title]

pattern_name = r"\s[–-]\s"
name = [re.split(pattern_name, i)[0] for i in name_title]
title = [re.split(pattern_name, i)[1] for i in name_title]

# some debugging:
title[1] = name[1].split(" / ")[1] + " - "+ title[1] 
name[1] = name[1].split(" / ")[0]
for i in [19, 31]:
    name[i] = name[i].split(",")[0]
name[20] = name[20].split(" PhD")[0]

In [41]:
#Checking

# print(len(name))
# print(len(title))
# print(name)
# print(title)

In [43]:
# Split: career summary, education

smmry = [re.split("\nEducation[^a-z]", ppl, flags = re.I)[0] for ppl in ppls_noname]
ppls_nosmy = [re.split("\nEducation[^a-z]", ppl, flags = re.I)[1] for ppl in ppls_noname]



education = [re.split("\n(\w* ){1,2}experience\s", ppl, flags = re.I)[0] for ppl in ppls_nosmy]

# test = [re.split("\n(\w* ){1,2}experience\s", ppl, maxsplit = 1, flags = re.I) for ppl in ppls_nosmy]
ppls_noedu = [re.split("\n(\w* ){1,2}experience\s", ppl, maxsplit = 1, flags = re.I)[2] for ppl in ppls_nosmy]

In [44]:
# Split: Work Experience
# 19 people has additional materials after work experience

pattern_work = r"\n(Selected )?Awards|\n(Selected\s)?Publications|\nPresentations|\nVideo\sPrograms"

# Selected Awards and Professional Achievements
# Slected Publications
# Presentations
# Publications and Awards
# Video Programs
# Awards
# Awards/Honors
# Publications
# Publications, Techical Reports, and Presentations
# Awards and Professional Achievements

ppls = [re.split(pattern_work, i) for i in ppls_noedu]
wrk = [i[0] for i in ppls]
ppls_others = [i[-1] if len(i) != 1 else None for i in ppls]

In [46]:
# #Split Work Experience

# # further seperate work experience
# pattern_exp = r"\n[^\n]+\d{4}\s?[–-]"
# wrk_exp_list = [re.split(pattern_exp, i) for i in wrk_exp] # a list of lists of work description
# wrk_exp_list1 = [re.findall(pattern_exp, i) for i in wrk_exp] # a list of lists of work_title/time

# #Check
# # print ([len(i) for i in wrk_exp_list])
# # print ([len(i) for i in wrk_exp_list1])

# for i in range(len(wrk_exp_list)):
#     # wrk_exp_list[i] is a list containing work experiences of i-th consultant
#     for j in range(len(wrk_exp_list1[i])):
#         wrk_exp_list[i][j+1] = wrk_exp_list1[i][j]+ wrk_exp_list[i][j+1]
        
#Check
# print(wrk_exp_list)
# print ([len(i) for i in wrk_exp_list])

In [51]:
newdata = [QEs[i]+data[i] for i in range(len(data))] 

In [54]:
# Build a simple dataframe to store the chunked information

df3 = pd.DataFrame({"Source": "B",
                    "Name":name,
                    "Title": title,
                    "Qualification and Experience": QEs,
                    "Career Summary": smmry,
                    "ETC": education,
                    "Work Experience": wrk,
                    "Hornors/Awards/Publications": ppls_others,
                    'ppls_data': newdata}) # for extract keywords, but repetive information (no name, title, QE)

In [56]:
#Storing the dataframe

file_name = data_folder / "intermediate/B_Phase1.pkl"
df3.to_pickle(file_name)